<a href="https://colab.research.google.com/github/sarikasea/Data-Science-Capstone/blob/main/Plotly_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install plotly dash

In [17]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2025-11-06 00:29:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-11-06 00:29:23 (856 MB/s) - ‘spacex_launch_dash.csv.1’ saved [2476/2476]



In [18]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"

--2025-11-06 00:29:23--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2075 (2.0K) [application/x-python]
Saving to: ‘spacex-dash-app.py.1’

spacex-dash-app.py. 100%[===================>]   2.03K  --.-KB/s    in 0s      

2025-11-06 00:29:23 (773 MB/s) - ‘spacex-dash-app.py.1’ saved [2075/2075]



In [19]:
!python3.11 spacex-dash-app.py

/bin/bash: line 1: python3.11: command not found


TASK 1: Add a Launch Site Drop-down Input Component


In [20]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Assuming you have loaded your spacex_df dataframe
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# Get unique launch sites from the dataframe
launch_sites = spacex_df['Launch Site'].unique()

# Create the Dash app
app = dash.Dash(__name__)

# Create the app layout
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'}
        ] + [
            {'label': site, 'value': site} for site in launch_sites
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # Add other components here (graphs, etc.)
])

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

In [21]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Assuming you have loaded your spacex_df dataframe
# spacex_df = pd.read_csv('your_spacex_data.csv')

# Get unique launch sites from the dataframe
launch_sites = spacex_df['Launch Site'].unique()

# Create the Dash app
app = dash.Dash(__name__)

# Create the app layout
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'}
        ] + [
            {'label': site, 'value': site} for site in launch_sites
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
])

# TASK 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Use all rows to render the pie chart for all sites
        fig = px.pie(
            spacex_df,
            values='class',
            names='Launch Site',
            title='Total Success Launches By Site'
        )
        return fig
    else:
        # Filter the dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

        # Group by class and count occurrences
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']

        # Create labels for success/failure
        class_counts['outcome'] = class_counts['class'].map({0: 'Failure', 1: 'Success'})

        # Render pie chart for the selected site
        fig = px.pie(
            class_counts,
            values='count',
            names='outcome',
            title=f'Total Success Launches for site {entered_site}'
        )
        return fig

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

TASK 3: Add a Range Slider to Select Payload


In [22]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Assuming you have loaded your spacex_df dataframe
# spacex_df = pd.read_csv('your_spacex_data.csv')

# Get unique launch sites from the dataframe
launch_sites = spacex_df['Launch Site'].unique()

# Get min and max payload values
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Create the Dash app
app = dash.Dash(__name__)

# Create the app layout
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'}
        ] + [
            {'label': site, 'value': site} for site in launch_sites
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={
            0: '0',
            2000: '2000',
            4000: '4000',
            6000: '6000',
            8000: '8000',
            10000: '10000'
        },
        value=[min_payload, max_payload]
    ),
    html.Br(),
])

# TASK 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Use all rows to render the pie chart for all sites
        fig = px.pie(
            spacex_df,
            values='class',
            names='Launch Site',
            title='Total Success Launches By Site'
        )
        return fig
    else:
        # Filter the dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

        # Count successes and failures
        success_count = filtered_df[filtered_df['class'] == 1].shape[0]
        failure_count = filtered_df[filtered_df['class'] == 0].shape[0]

        # Create a dataframe for the pie chart
        outcome_df = pd.DataFrame({
            'outcome': ['Success', 'Failure'],
            'count': [success_count, failure_count]
        })

        fig = px.pie(
            outcome_df,
            values='count',
            names='outcome',
            title=f'Total Success Launches for site {entered_site}'
        )
        return fig

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

In [24]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Assuming you have loaded your spacex_df dataframe
# spacex_df = pd.read_csv('your_spacex_data.csv')

# Get unique launch sites from the dataframe
launch_sites = spacex_df['Launch Site'].unique()

# Get min and max payload values
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Create the Dash app
app = dash.Dash(__name__)

# Create the app layout
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'}
        ] + [
            {'label': site, 'value': site} for site in launch_sites
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={
            0: '0',
            2000: '2000',
            4000: '4000',
            6000: '6000',
            8000: '8000',
            10000: '10000'
        },
        value=[min_payload, max_payload]
    ),
    html.Br(),

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(
            spacex_df,
            values='class',
            names='Launch Site',
            title='Total Success Launches By Site'
        )
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']
        class_counts['outcome'] = class_counts['class'].map({0: 'Failure', 1: 'Success'})
        fig = px.pie(
            class_counts,
            values='count',
            names='outcome',
            title=f'Total Success Launches for site {entered_site}'
        )
        return fig

# TASK 4: Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]

    if entered_site == 'ALL':
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Correlation between Payload and Success for all Sites'
        )
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Correlation between Payload and Success for site {entered_site}'
        )
        return fig

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [25]:
# Which site has the largest successful launches?
successful_launches_by_site = spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts()
largest_successful_launches_site = successful_launches_by_site.idxmax()
print(f"Site with the largest successful launches: {largest_successful_launches_site}")

# Which site has the highest launch success rate?
launch_site_success_rate = spacex_df.groupby('Launch Site')['class'].mean().sort_values(ascending=False)
highest_success_rate_site = launch_site_success_rate.index[0]
print(f"Site with the highest launch success rate: {highest_success_rate_site} ({launch_site_success_rate.iloc[0]:.2f})")

# Which payload range(s) has the highest launch success rate?
# Define payload ranges (you can adjust these as needed)
payload_bins = [0, 2000, 4000, 6000, 8000, 10000]
payload_labels = ['0-2000', '2000-4000', '4000-6000', '6000-8000', '8000-10000']
spacex_df['Payload Range'] = pd.cut(spacex_df['Payload Mass (kg)'], bins=payload_bins, labels=payload_labels, right=False)

payload_success_rate = spacex_df.groupby('Payload Range')['class'].mean().sort_values(ascending=False)
highest_success_rate_payload = payload_success_rate.index[0]
print(f"Payload range with the highest launch success rate: {highest_success_rate_payload} ({payload_success_rate.iloc[0]:.2f})")

# Which payload range(s) has the lowest launch success rate?
lowest_success_rate_payload = payload_success_rate.index[-1]
print(f"Payload range with the lowest launch success rate: {lowest_success_rate_payload} ({payload_success_rate.iloc[-1]:.2f})")


# Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
booster_success_rate = spacex_df.groupby('Booster Version Category')['class'].mean().sort_values(ascending=False)
highest_success_rate_booster = booster_success_rate.index[0]
print(f"Booster version with the highest launch success rate: {highest_success_rate_booster} ({booster_success_rate.iloc[0]:.2f})")

Site with the largest successful launches: KSC LC-39A
Site with the highest launch success rate: KSC LC-39A (0.77)
Payload range with the highest launch success rate: 2000-4000 (0.62)
Payload range with the lowest launch success rate: 6000-8000 (0.00)
Booster version with the highest launch success rate: B5 (1.00)


/tmp/ipython-input-2980340139.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

